In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 3.4 MB/s eta 0:00:00


In [3]:
import praw
import json
import pandas as pd
import prawcore  # Import prawcore to handle specific exceptions


# Initialize the Reddit instance with credentials (replace with your own)
reddit = praw.Reddit(
    client_id="wGZwffGplDx3CfTr-ivI3Q",
    client_secret="JRZr87__rHWLcQFDtQlgE1btNZ9Rlw",
    user_agent="MedicationMentionsScraper 1.0 by /u/your_username"
)

# Define the list of psychiatric medications to search for
psychiatric_medications =  ["Vyvanse","Concerta","Intuniv","methylphenidate","Strattera"]


# Define a list of subreddits where users frequently discuss health topics
subreddits = ["AskReddit", "mentalhealth", "adhd", "psychiatry", 'medication', 'antidepressants', 'askdocs']

# Function to search posts related to the medications
def search_for_posts():
    results = []

    for medication in psychiatric_medications:
        for subreddit in subreddits:
            print(f"Searching in {subreddit} for posts mentioning {medication}...")

            try:
                # Search for posts that mention the medication
                for submission in reddit.subreddit(subreddit).search(medication, limit=500):  # Adjust limit as needed
                    data = {
                        'title': submission.title,
                        'text': submission.selftext,
                        'medication': medication,
                        'subreddit': subreddit,
                        'upvotes': submission.score,
                        'author': str(submission.author)
                    }
                    results.append(data)
                    print(f"Found post: {submission.title}")

                # Search through comments in each subreddit
                for submission in reddit.subreddit(subreddit).search(medication, limit=5):  # Searching posts to find comments
                    submission.comments.replace_more(limit=0)
                    for comment in submission.comments.list():
                        data = {
                            'comment': comment.body,
                            'medication': medication,
                            'subreddit': subreddit,
                            'upvotes': comment.score,
                            'author': str(comment.author)
                        }
                        results.append(data)
                        print(f"Found comment: {comment.body[:50]}...")  # Print first 50 characters of the comment

            except prawcore.exceptions.NotFound:
                print(f"404 error encountered in {subreddit} for medication {medication}. Skipping.")
                continue
            except praw.exceptions.PRAWException as e:
                print(f"Error encountered in {subreddit} for medication {medication}: {e}")
                continue

    return results

# Call the function to search for posts and comments
medication_data = search_for_posts()

# Save the results to a JSON file
with open("scrape_adhd.json", "w") as f:
    json.dump(medication_data, f, indent=4)

print(f"Scraping complete. Found {len(medication_data)} posts/comments.")

Searching in AskReddit for posts mentioning Vyvanse...
Found post: What was that "one thing" that made weight loss finally work for you?

Found post: People that sleep less than 6 hours a day, how do you function ?
Found post: People with ADHD, how would you describe ADHD to someone who knows nothing about it?
Found post: What's your best college-budget meal?
Found post: How did Vyvanse change your relationship with your friends?
Found post: What changed your life forever?
Found post: Anyone with experience taking long term psych meds such as daily Vyvanse/Adderall how has this affected you over the course of decades? Does it age you like street methamphetamine would?
Found post: for people in the US (with or without insurance) what does your prescription of vyvanse cost you?
Found post: Ex-Procrastinators of Reddit, what motivated you to bring about a change in your ways? [SERIOUS]
Found post: For those who take adderall/vyvanse do you think it has done more good than harm or vice ver